In [2]:
pip install cmake scipy


In [3]:
pip install gym

  Created wheel for gym: filename=gym-0.17.2-py3-none-any.whl size=1650896 sha256=eb1985827be7434d69e5a5606e17d27c2bfa53348f9761b9b92ac3e741349b6a
  Stored in directory: c:\users\io\appdata\local\pip\cache\wheels\48\bf\7c\44b1b8e4ad998fc48e31caedbb9e028351861b8d20632642bc
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491062 sha256=e03a852655a7b966a43506e826a85789e5f538dbcc505fb2c0f80bcfe95f0008
  Stored in directory: c:\users\io\appdata\local\pip\cache\wheels\8e\70\28\3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built gym future



In [12]:
import gym

In [11]:
env = gym.make("Taxi-v3").env
env.render()


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [14]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [15]:
print(f"action space{env.action_space}")
print(f"state space{env.observation_space}")

action spaceDiscrete(6)
state spaceDiscrete(500)


In [16]:
state = env.encode(1,2,2,0)
print("state",state)
env.s= state
env.render()

state 148
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [17]:
env.P[148]

{0: [(1.0, 248, -1, False)],
 1: [(1.0, 48, -1, False)],
 2: [(1.0, 168, -1, False)],
 3: [(1.0, 148, -1, False)],
 4: [(1.0, 148, -10, False)],
 5: [(1.0, 148, -10, False)]}

In [18]:
import numpy as np

In [20]:
q_table  =np.zeros([env.observation_space.n,env.action_space.n])

In [21]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

Wall time: 1min 46s


In [22]:
q_table[148]

array([ -2.3639511 ,  -2.43470686,  -2.44145024,  -2.4155152 ,
       -10.5368024 , -11.09190341])

In [23]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.09
Average penalties per episode: 0.0
